In [ ]:
%%capture
# @title Setup Environment
# Install the latest Tensorflow version.
# !pip install -q "transformers==4.9.2"
# !pip install -q "datasets==1.11.0"

In [24]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from ast import literal_eval

In [ ]:
#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def bert_model(question, context):
    encoding = tokenizer.encode_plus(text=question,text_pair=context)
    inputs = encoding['input_ids']  #Token embeddings
    sentence_embedding = encoding['token_type_ids']  #Segment embeddings
    tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

    output = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
    start_index = torch.argmax(output.start_logits)

    end_index = torch.argmax(output.end_logits)

    answer = ' '.join(tokens[start_index:end_index+1])

    corrected_answer = ''

    for word in answer.split():
    
        #If it's a subword token
        if word[0:2] == '##':
            corrected_answer += word[2:]
        else:
            corrected_answer += ' ' + word
    

    return corrected_answer

In [ ]:
cont_file = "output/resume_content.txt"

In [ ]:
edu_out = open("output/education.txt", 'w+')
with open(cont_file, 'r') as f:
    
    for line in f:
        content = list(literal_eval(line))
        for sec in content:
            
            title = sec['title']
            id = sec['id']
            
            if "education" in title :
                
                ques_college = "what is your college?"
                ques_major = "what is your major?"
                ques_degree = "what is your degree"

                context = ' '.join(sec['content'])
                college = bert_model(ques_college, context)
                major = bert_model(ques_major, context)
                degree = bert_model(ques_degree, context)

                print('{}, {}, {},{}, [{}]\n'.format(id, college, major, degree, context))
                edu_out.write('{}, {}, {}, {}, [{}]\n'.format(id, college, major, degree, context))


 mount vernon nazarene university
 business administration
 bachelor of business administration
16852973,  mount vernon nazarene university,  business administration, bachelor of business administration, [Bachelor of Business Administration MOUNT VERNON NAZARENE UNIVERSITY Ôºç City , State]



In [45]:
import json

exp_out = open("output/exp_summary.txt", 'w+')

expr_file = "output/experiences.txt"
with open(expr_file, 'r') as f:
    
    for line in f:
        exps = json.loads(line)
        id = exps['id']

        wkex = "I have "+str(len(exps['experiences']))+ " work experiences. "
        for exp in exps['experiences']:
            jobtitle = exp['jobtitle']
            startsdate = exp['startsdate']
            endsdate = exp['endsdate']
            if exp['jobduty']:
                jobduty = '. '.join(exp['jobduty'])

            if endsdate:
                wkex += "As a "+jobtitle+", I worked from " + startsdate + " to " + endsdate + ". And my major duty is "+jobduty
            elif startsdate:
                 wkex += "As a "+jobtitle+", I worked from " + startsdate + " to now. And my major duty is "+jobduty
        exp_out.write("{}|{}\n".format(id, wkex))
        

{'id': '16852973', 'experiences': [{'jobtitle': 'HR Administrator/Marketing Associate  HR Administrator', 'startsdate': 'Dec 2013', 'endsdate': 'Current', 'jobduty': ['Helps to develop policies, directs and coordinates activities such as employment, compensation, labor relations, benefits, training, and employee services.', 'Prepares employee separation notices and related documentation', 'Keeps records of benefits plans participation such as insurance and pension plan, personnel transactions such as hires, promotions, transfers, performance reviews, and terminations, and employee statistics for government reporting.', 'Advises management in appropriate resolution of employee relations issues.', 'Administers benefits programs such as life, health, dental, insurance, pension plans, vacation, sick leave, leave of absence, and employee assistance.', 'Marketing Associate', 'Designed and created marketing collateral for sales meetings, trade shows and company executives.', 'Managed the in-h